In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: no se puede crear el directorio «build»: El archivo ya existe
-- The C compiler identification is GNU 8.3.0
-- The CXX compiler identification is GNU 8.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /usr/bin/python (found version "2.7.16") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/guidotripodi/Documentos/Facu/MN/mntp2/mnv-tp2
-- Configuring done


In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: No se puede efectuar open: No existe el archivo o el directorio
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [3]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

# Experimento KNN SIN PCA


In [5]:
import sentiment
from sklearn.metrics import accuracy_score

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))

for k in k_values:

    clf = sentiment.KNNClassifier(k)

    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    print("k: {}   acc: {}".format(k, acc))
    scores.append(acc)
    
print(scores)

k 1 :  acc 0.5499601593625498
k 5 :  acc 0.5915537848605578
k 10 :  acc 0.6065338645418327
k 15 :  acc 0.6237450199203187
k 20 :  acc 0.6341035856573706
k 25 :  acc 0.6341035856573706
k 50 :  acc 0.6473306772908366
k 100 :  acc 0.6388844621513944
k 150 :  acc 0.6283665338645419
k 200 :  acc 0.6224701195219123
k 250 :  acc 0.6224701195219123
k 300 :  acc 0.6167330677290837
k 350 :  acc 0.6162549800796813
[0.5499601593625498, 0.5915537848605578, 0.6065338645418327, 0.6237450199203187, 0.6341035856573706, 0.6341035856573706, 0.6473306772908366, 0.6388844621513944, 0.6283665338645419, 0.6224701195219123, 0.6224701195219123, 0.6167330677290837, 0.6162549800796813]


# Experimento KNN variando max y min df

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score

maxDF_values = [0.80,0.85,0.90,0.92,0.95,1.00]
minDF_values = [0.01,0.02,0.03,0.05,0.08,0.10]

for maxDF in maxDF_values:
    
    for minDF in minDF_values:
        
        vectorizer = CountVectorizer(max_df=maxDF, min_df=minDF, max_features=5000)

        vectorizer.fit(text_train)

        X_train1, y_train1 = vectorizer.transform(text_train), (label_train == 'pos').values
        X_test1, y_test1 = vectorizer.transform(text_test), (label_test == 'pos').values

        scores = []
        k_values = [50]

        for k in k_values:
            clf = sentiment.KNNClassifier(k)
            clf.fit(X_train1, y_train1)
            y_pred1 = clf.predict(X_test1)
            acc = accuracy_score(y_test1, y_pred1)
            print("k: {}   acc: {} maxDF: {} minDF: {}".format(k, acc,maxDF, minDF))
            scores.append(acc)
    
print(scores)

k: 50   acc: 0.6643824701195219 maxDF: 0.8 minDF: 0.01
k: 50   acc: 0.6624701195219124 maxDF: 0.8 minDF: 0.02
k: 50   acc: 0.6616733067729084 maxDF: 0.8 minDF: 0.03
k: 50   acc: 0.6661354581673307 maxDF: 0.8 minDF: 0.05
k: 50   acc: 0.6541832669322709 maxDF: 0.8 minDF: 0.08
k: 50   acc: 0.6567330677290837 maxDF: 0.8 minDF: 0.1
k: 50   acc: 0.6621513944223107 maxDF: 0.85 minDF: 0.01
k: 50   acc: 0.6613545816733067 maxDF: 0.85 minDF: 0.02
k: 50   acc: 0.660398406374502 maxDF: 0.85 minDF: 0.03
k: 50   acc: 0.6599203187250996 maxDF: 0.85 minDF: 0.05
k: 50   acc: 0.6535458167330678 maxDF: 0.85 minDF: 0.08
k: 50   acc: 0.6513147410358566 maxDF: 0.85 minDF: 0.1
k: 50   acc: 0.6473306772908366 maxDF: 0.9 minDF: 0.01
k: 50   acc: 0.6438247011952192 maxDF: 0.9 minDF: 0.02
k: 50   acc: 0.6441434262948207 maxDF: 0.9 minDF: 0.03
k: 50   acc: 0.6334661354581673 maxDF: 0.9 minDF: 0.05
k: 50   acc: 0.6326693227091633 maxDF: 0.9 minDF: 0.08
k: 50   acc: 0.6310756972111554 maxDF: 0.9 minDF: 0.1
k: 50   

# Experimento KNN CON PCA

In [13]:
import sentiment
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.85, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train2, y_train2 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test2, y_test2 = vectorizer.transform(text_test), (label_test == 'pos').values

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))
alpha_values = [50,100,150]

for alpha in alpha_values:
    for k in k_values:
        X_train2aux = X_train2
        pca = sentiment.PCA(alpha)
        pca.fit(X_train2aux.toarray())
        
        X_train2aux = pca.transform(X_train2aux)
        X_test2 = pca.transform(X_test2)
        
        clf = sentiment.KNNClassifier(k)
        clf.fit(X_train2aux, y_train2)
        
        y_pred2 = clf.predict(X_test2)
        acc = accuracy_score(y_test2, y_pred2)
        print("k: {}  acc: {} Alpha: {} ".format(k, acc, alpha))
        scores.append(acc)

print(scores)

k: 1  acc: 0.5062948207171315 Alpha: 50 
k: 5  acc: 0.49338645418326693 Alpha: 50 
k: 10  acc: 0.49673306772908365 Alpha: 50 
k: 15  acc: 0.5064541832669323 Alpha: 50 
k: 20  acc: 0.5064541832669323 Alpha: 50 
k: 25  acc: 0.5064541832669323 Alpha: 50 
k: 50  acc: 0.5064541832669323 Alpha: 50 
k: 100  acc: 0.5064541832669323 Alpha: 50 
k: 150  acc: 0.49354581673306774 Alpha: 50 
k: 200  acc: 0.49354581673306774 Alpha: 50 
k: 250  acc: 0.49354581673306774 Alpha: 50 
k: 300  acc: 0.49354581673306774 Alpha: 50 
k: 350  acc: 0.49354581673306774 Alpha: 50 
k: 1  acc: 0.49354581673306774 Alpha: 100 
k: 5  acc: 0.5064541832669323 Alpha: 100 
k: 10  acc: 0.5064541832669323 Alpha: 100 
k: 15  acc: 0.5064541832669323 Alpha: 100 
k: 20  acc: 0.5064541832669323 Alpha: 100 
k: 25  acc: 0.5064541832669323 Alpha: 100 


KeyboardInterrupt: 

# Variando cantidad de instancias de entrenamiento KNN SIN PCA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score

vectorizer = CountVectorizer(max_df=0.85, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train3, y_train3 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test3, y_test3 = vectorizer.transform(text_test), (label_test == 'pos').values

x_values = [5,10,20,50,100,500,1000,2000,3000,4000,5000,6500]

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))

for x in x_values:
    
    X_train3 = X_train3[0:x]
    y_train3 = y_train3[0:x]
        
    for k in k_values:

        clf = sentiment.KNNClassifier(k)

        clf.fit(X_train3, y_train3)

        y_pred3 = clf.predict(X_test3)

        acc = accuracy_score(y_test3, y_pred3)
        print("k: {}   acc: {}".format(k, acc))
        scores.append(acc)
    
print(scores)

# Variando cantidad de instancias de entrenamiento KNN CON PCA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score

vectorizer = CountVectorizer(max_df=0.85, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train4, y_train4 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test4, y_test4 = vectorizer.transform(text_test), (label_test == 'pos').values

x_values = [5,10,20,50,100,500,1000,2000,3000,4000,5000,6500]

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))
alpha_values = [50,100,150]
for x in x_values:
    
    X_train4 = X_train4[0:x]
    y_train4 = y_train4[0:x]
    for alpha in alpha_values:
        for k in k_values:
            X_train4aux = X_train4
            pca = sentiment.PCA(alpha)
            pca.fit(X_train4aux.toarray())

            X_train4aux = pca.transform(X_train4aux)
            X_test4 = pca.transform(X_test4)

            clf = sentiment.KNNClassifier(k)
            clf.fit(X_train4aux, y_train4)

            y_pred4 = clf.predict(X_test4)
            acc = accuracy_score(y_test4, y_pred4)
            print("k: {}   acc: {}  Alpha: {}".format(k, acc, alpha))
            scores.append(acc)

print(scores)